<a href="https://colab.research.google.com/github/MHoseinHoushmand/Clustering_by_SLFA/blob/main/Clustering_by_SLFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np

import pdb
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)


labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data)} documents - {true_k} categories")

3387 documents - 4 categories


In [42]:

def docs_as_tfidf(docs):
  vectorizer = TfidfVectorizer(
     max_df=0.5,
     min_df=5,
     stop_words="english",
  )

  docs_vector = vectorizer.fit_transform(docs)
  return docs_vector.toarray()



In [43]:
population_size = 120 # Frogs number
memplex_num = 12 #define as m
memplex_size = 10 #define as n
max_iteration = 50
memplex_iteration = 8
docs = dataset.data
docs_vector = docs_as_tfidf(docs)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
def cosin_sim(a,b):
   return cosine_similarity([a], [b])[0][0]

In [45]:
from numpy.linalg import norm
def SSE(cluster,doc_mean):
  size = len(cluster)
  sse=0
  for doc in cluster:
    sse += cosin_sim(doc,doc_mean)**2
  sse = sse/size
  return sse

In [46]:
from numpy.linalg import norm
def BC(doc_means):
   BC=0
   size = len(doc_means)
   for i in range(size):
      for j in range(i+1,size):
          BC += cosin_sim(doc_means[i],doc_means[j])**2
   return BC

In [47]:
def WC(clusters):
    WC = 0
    for cluster in clusters:
        doc_mean = np.average(cluster, axis=0)
        WC += SSE(cluster,doc_mean)
    return WC


In [48]:
def build_clusters(answer,docs_vector,clusters_size):
   clusters = []
   for i in range(clusters_size):
       clusters.append([])
   for j in range(len(answer)):
       if -1 < answer[j]:
        clusters[answer[j]].append(docs_vector[j])
   return clusters

In [49]:
def fitness(answer,docs_vector,clusters_size):
   doc_means = []
 #  pdb.runcall(build_clusters,answer,docs_vector,clusters_size)
   clusters = build_clusters(answer,docs_vector,clusters_size)
   for i in range(clusters_size):
       doc_means.append(
          np.average(clusters[i], axis=0)
       )
   wc = WC(clusters)
   bc = BC(doc_means)
   fitness = wc/bc
   return fitness


In [50]:
import random
def cross_over(answer_a,answer_b):
    size = len(answer_a)
    output = []
    for i in range(size):
       choice = random.choice([0,1])
       if choice == 0:
          output.append(answer_a[i])
       else:
          output.append(answer_b[i])
    return tuple(output)

In [51]:
def best_and_worst(answers):
     best =  max(answers, key=answers.get)
     worst = min(answers, key=answers.get)
     return tuple(best) , tuple(worst)

In [52]:
def global_best(memplexes):
     local_bests = {}
     for memplex in memplexes:
         local_best =  max(memplex, key=memplex.get)
         local_bests[local_best]= memplex[local_best]
     global_best = max(local_bests, key=local_bests.get)
     return max(global_best, local_bests[global_best])

In [53]:
def keys_to_remove(keys , dict):
   for k in keys:
      if k in dict:
          dict.pop(k)
   return dict

In [54]:
def mutation(global_best,clusters_size):
    new_ans = global_best
    size = len(global_best)/4
    array = np.random.choice(np.arange(-1,clusters_size), size=size, replace=False)
    for i in array:
      new_ans = new_ans[i]
    return new_ans

In [55]:
def Create_memplexes(population,memplex_num):
     memplexes = []
     keys = list(population.keys())
     population_size = len(population)
     for i in range(memplex_num):
         memplexes.append({})
     for i in range(population_size):
         memplexes[i % memplex_num][keys[i]] = population[keys[i]]
     return memplexes


In [ ]:
def sumlists(left, right):
   if (len(left) == 1 and len(right) == 0) :
      return left[0]
   if (len(left) == 1 and len(right) == 1) :
      return left[0].update(right[0])
   if (len(left) == 0 and len(right) == 1) :
      return right[0]
   else :
      return sumlists(left[0:int(len(left)/2)], left[int(len(left)/2)+1:len(left)]).update(sumlists(right[0:int(len(right)/2)], right[int(len(right)/2)+1:len(right)]))

In [57]:
def shufeling(memplexes):
    size = len(memplexes)
    half_size = int(size/2)
    left = memplexes[0:half_size]
    right = memplexes[half_size+1:size]
    output =sumlists(left, right)
    return output




In [60]:
import operator
def frog_leaping_search(docs_vector,cluster_size):
     answers=np.random.randint(-1, cluster_size, size=(population_size , len(dataset.data)))
     population = {}
     i=0
     for answer in answers:
       # pdb.runcall(fitness,answer,docs_vector,cluster_size)
        i+=1
        population[tuple(answer)] = fitness(answer,docs_vector,cluster_size)
        print(i,population[tuple(answer)])
     population = dict( sorted(population.items(), key=operator.itemgetter(1), reverse=True))
   #  pdb.runcall(Create_memplexes,population, memplex_num)
     memplexes = Create_memplexes(population, memplex_num)
     population.clear()
     for i in range(max_iteration):
         pdb.set_trace()
         for j in range(memplex_num):
            sub_memplex = dict(random.sample(list(memplexes[j].items()),k=5))
            memplexes[j] =  keys_to_remove(sub_memplex.keys(),memplexes[j])
            for k in range(memplex_iteration):
               #pdb.runcall(best_and_worst,sub_memplex)
               ans_best, ans_worst = best_and_worst(sub_memplex)
               ans_out = cross_over(ans_best,ans_worst)
               fitness_out = fitness(ans_out,docs_vector,cluster_size)

              # pdb.runcall(best_and_worst,sub_memplex)
               #print(population[ans_worst],population[ans_out])
               if (sub_memplex[ans_worst]<fitness_out):
                    del sub_memplex[ans_worst]
                    sub_memplex[ans_out] = fitness_out
               else:
                   # pdb.runcall(global_best,memplexes)
                    g_best, g_value = global_best(memplexes)
                    ans_out = cross_over(g_best,ans_worst)
                    fitness_out = fitness(ans_out,docs_vector,cluster_size)
                    if (sub_memplex[ans_worst] < fitness_out):
                        del sub_memplex[ans_worst]
                        sub_memplex[ans_out] = fitness_out
                    else:
                        del sub_memplex[ans_worst]
                        pdb.runcall(mutation,g_best,cluster_size)
                        ans_out = mutation(g_best,cluster_size)
                        fitness_out = fitness(ans_out,docs_vector,cluster_size)
                        sub_memplex[ans_out] = fitness_out
        #    pdb.runcall(join_dicts,memplexes[j],sub_memplex)
            memplexes[j].update(sub_memplex)









In [59]:
frog_leaping_search(docs_vector,4)

1 0.014081682610111704
2 0.013948851827504438
3 0.014096361262124452
4 0.01416415932628135
5 0.013896217385707512
6 0.014057967388237032
7 0.013992308612572306
8 0.013920160783240891
9 0.014011537527874245
10 0.013801842869426515
11 0.013924121669851798
12 0.014069060025897886
13 0.013845942502030728
14 0.01382029003111502
15 0.014047376018870588
16 0.01421961306551745
17 0.01409838136855702
18 0.014151000076084962
19 0.013806196387040292
20 0.013928045526791782
21 0.01396723759678985
22 0.014078584954716788
23 0.014020538717889067
24 0.014300486712991702
25 0.013995222007630356
26 0.013987003012782995
27 0.014211881333692219
28 0.014345096869169825
29 0.013865110258340686
30 0.014316623899651755
31 0.013979531852908645
32 0.01395320210354171
33 0.014035433458457832
34 0.014210905909605583
35 0.013798840093869034
36 0.013980661762595605
37 0.014013877246735095
38 0.014037566957694743
39 0.01390422092308328
40 0.0139885753348081
41 0.013993375197430014
42 0.014126746782764479
43 0.01397

ValueError: ignored

In [ ]:
uniqueList=[[3,2,1,3],[5,3,2,1],[3,8,5,9],[2,6,4,9]]

print(random.sample(uniqueList, 2))

a= {'s':3,'4':6,'sdgfsdfg':656}
b= max(a.values())
print(b)

In [ ]:
import operator
x = {'4':7,'8':2,'99':45}
x = dict( sorted(x.items(), key=operator.itemgetter(1), reverse=True))
print (x)

In [ ]:

import numpy as np

# تعیین بازه مقداری

import pdb
beeeeee =46363
zee={'ddd':2352}
for i in range(1):
   pdb.set_trace()
   array = np.random.choice(np.arange(-1, 10), size=5, replace=False)
   print(array)

ggggga=643
print('nali')


In [ ]:
import random

my_dict = {'a': 1.235, 'b': 2.23525235, 'c': 3.235223523, 'd': 4.235235235, 'e': 5.23525252323}

del my_dict[['a','b']]
random_objects = random.sample(list(my_dict.items()), k=3)
print(random_objects)

